# <font color = 'red'> Analisando Sentimentos das reclamações</font>

In [1]:
!pip install -q textblob

In [27]:
!pip install transformers

     --------------------------------------- 10.0/10.0 MB 13.1 MB/s eta 0:00:00
     -------------------------------------- 447.5/447.5 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 13.8 MB/s eta 0:00:00
     -------------------------------------- 286.1/286.1 kB 8.6 MB/s eta 0:00:00
     -------------------------------------- 179.6/179.6 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [28]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModel
import torch

from textblob import TextBlob

import warnings
warnings.filterwarnings('ignore')

In [39]:
dados_sentimento = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',')

In [40]:
dados_sentimento.sample(5)

,Empresa,Relato,Avaliacao
1354,Banco Inter (Banco Intermedium),tenho uma divida com um cartao de credito e su...,4
1403,Banco do Brasil,pretendo renegociar a pendência. porém com os ...,1
205,Caixa Econômica Federal,29102024 1641 55 21 xxxxxx981 caro a aluno a ...,1
1344,Caixa Econômica Federal,informo que a caixa econômica não reduz os jur...,2
1609,Banco do Brasil,já fui na agência do banco do brasil duas veze...,1


In [41]:
dados_sentimento.shape

(1920, 3)

In [35]:
### Função para obter subjetividade
def subjetividade(text):
 blob = TextBlob(text)
 return blob.sentiment.subjectivity

In [36]:
# Função para obter polaridade
def polaridade(text):
 blob = TextBlob(text)
 return blob.sentiment.polarity

In [37]:
# Adicionando no DF
dados_sentimento['Subjetividade'] = dados_sentimento['Relato'].apply(subjetividade)
dados_sentimento['Polaridade'] = dados_sentimento['Relato'].apply(polaridade)

In [42]:
dados_sentimento.head(10)

,Empresa,Relato,Avaliacao
0,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3
1,Nubank,ja houve o pagamento da negociacao,2
2,Nubank,exclusao do nome retirar negativacao do nome s...,1
3,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1
4,Nubank,gostaria de reclamar sobre essa dívida quer es...,5
5,Banco Itaú Unibanco,o itaú financiamentos fez constar na scr uma l...,1
6,Nubank,eu fiz um acordo para pagamento da minha dívid...,3
7,Banco Itaú Unibanco,primeiramente quero agradecer o banco itau poi...,5
8,Nubank,reclamação nubank gostaria de registrar minha ...,1
9,Caixa Econômica Federal,desejo participar do mutirão de renegociação.,1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar o modelo pré-treinado para análise de sentimento
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Função para prever a polaridade e intensidade com RoBERTa-BR
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    scores = outputs.logits.softmax(dim=1).detach().numpy()
    
    # Exemplos de interpretação (ajustar conforme os rótulos do modelo que está usando)
    sentiment = "positivo" if scores[0][1] > scores[0][0] else "negativo"
    intensity = scores[0][1]  # Considerando que pontuações mais altas indicam mais intensidade
    
    return sentiment, intensity

# Aplicar o modelo para obter polaridade e intensidade
dados_sentimento['Polaridade'], dados_sentimento['Intensidade'] = zip(*dados_sentimento['Relato'].apply(get_sentiment))

# Exibir o resultado
print(dados_sentimento)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
